Notebook authored by Bowen Jiang

In [1]:
#Basics 
import numpy as np
import pandas as pd 
import math, random
import sys, os 
import sklearn
from scipy import stats 
import matplotlib 
import matplotlib.pyplot as plt 
%matplotlib inline

#Google Drive setup 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#Navigate to directory with Google Drive
%cd /content/drive/Shareddrives/CS\ 272\ PCOS\ FL/Synthetic\ Data\ Generation\ /

/content/drive/Shareddrives/CS 272 PCOS FL/Synthetic Data Generation 


In [3]:
# Dictionary of hormone distributions
# Mean, SD
pcos_hormones = {
    "lh_fsh_ratio" : (1.7,0.7),
    "estradiol" : (87.6,21.1),
    "testosterone" : (71.4,27.9),
    "progesterone_17oh" : (1.6,1),
    "dheas" : (777.5,1135.8),
    "androstenedione" : (5.2,4.3),
    "amh" : (76.0, 36.3)
}

normal_hormones = { ##Not used in this iteration of the code
    "lh_fsh_ratio" : (1.1,0.6),
    "estradiol" : (69.0,27.4),
    "testosterone" : (42.7,15.6),
    "progesterone_17oh" : (1.5,0.6),
    "dheas" : (1278.4,471.5),
    "androstenedione" : (1.4,0.5),    
}

In [4]:
"""
************* KNOBS FOR ML LEARNING: *****************
  - QUANTILE_WIDTH: This sets the minimum quantile from which to sample response type means.
  For example, QUANTILE_WIDTH = 0 and bins=10 means that one response type will have a mean 
  drawn from 0-10% lowest decile of the distribution; one mean drawn from 10-20% decile; etc.
  Setting quantile width higher (~0.4 for example) means that the maximal "spread" of hormone
  metric means will be narrower, making learning more challenging as the model will have to 
  distinguish between subtle differences in the distribution of hormones.

  -RESAMPLING_CONSTANT: This value is a measure of the "narrowness" of the standard deviations 
  of hormone metrics. Setting this number higher means that the hormone metrics sampled for 
  each patient response type will have less spread - coupled with a small value of QUANTILE_WIDTH,
  this can result in very separate distributions of hormone values per patient response type, making
  learning very easy. Note that RESAMPLING_CONSTANT should be at least 3 or so to prevent the 
  generation of too many negative hormone values (due to the skewedness of those distributions 
  and our use of the multivariate normal to generate data - it's exceedingly hard, i.e. statistics
  publication level hard, to generate data with a desired mean, covariance structure, and non-normal
  structure with a specified minimum value of 0). 

  -COVARIANCE_STRENGTH: A scaling factor for the off-diagonal elements in the covariance matrix. 
  Small values of this constant result in minimal covariance in the generated data, which may 
  make learning easier by maximizing the informativeness of each variable. Large values of this
  constant could make learning impossible. 

  -PROPORTION_TRUE_TREATMENT: The proportion of each patient response type that gets assigned the
  "true" OCP medication. What we attempt to model here is the fact that, even if patients have a
  feature profile that predisposes them to a certain most effective medication, their doctors may
  not prescribe it to them, or they are still in the process of trying out other medications, or
  there is actually a different medication that is most effective for them (due to some unmeasured
  covariates). Setting this proportion to 1 means that every patient of a given response type 
  has the same (successful) medication prescribed to them. Setting this proportion to 0 means that
  within this response type, there are 10 groups of ~10% of the patients, each group being prescribed
  a different most effective medication. Ultimately, this proportion controls the maximum expected
  accuracy of our model: acc_max = PROPORTION_TRUE_TREATMENT + 0.1*(1-PROPORTION_TRUE_TREATMENT)

  -PER_CLINIC_NOISE_SPREAD: A scaling factor for how much spread in the noise amounts across all
  of the different clinics. The idea is to generate some amount of normally-distributed, 
  0-mean noise to each measurement, where the SD is some multiple of the mean measurement whose 
  magnitude is controlled by the HORMONE_MEASUREMENT_NOISE constant. Can also control this distribution
  by re-tuning the gamma distribution parameters in the appropriate cell. 

  -HORMONE_MEASUREMENT_NOISE: A scaling factor for how much noise to add to each hormone measurement. 

  -SYMPTOM_MASK_CONST: A scaling factor for how much spread in the symptom-masking rate across all
  of the different clinics. Can also control this distribution by re-tuning the gamma distribution 
  parameters in the appropriate cell. 
"""
QUANTILE_WIDTH = 0.30 #Values in range [0, 0.5) -> lower number = easier learning
RESAMPLING_CONSTANT = 5 #Values in range [1, +inf] -> higher number = easier learning 
COVARIANCE_STRENGTH = 1 #Values in range (0, +inf) but best to keep around/below 1 -> lower number = easier learning 
PROPORTION_TRUE_TREATMENT = 0.8 #Values in range [0, 1] -> higher number = easier learning 
PER_CLINIC_NOISE_SPREAD = 1 #Values in range [0, +inf) -> lower number = easier learning 
HORMONE_MEASUREMENT_NOISE = 1 #Values in range [0, +inf) -> lower number = easier learning 
SYMPTOM_MASK_CONST = 5 #

In [5]:
# Set random seed for downstream!
np.random.seed(30)
hormone_ranges = {}

# Generate the ranges for each hormone
for hormone, dist in pcos_hormones.items():
    mean, sd = dist
    # Get the number of ranges to create
    hormone_num_ranges = np.random.randint(5,11)
    print(f"Number of ranges for {hormone}: "+str(hormone_num_ranges))

    #Set boundaries for each patient response type mean
    """
    Long comment - we really don't want negative hormone measurements as they
    don't make biological sense. However, there's really no easy way to parametrically 
    generate correlated, multivariate datapoints that are non-normal (i.e. we 
    can specify a minimum value of 0, a desired mean and a standard deviation 
    for each variaable/feature along with inter-variable correlation). If we 
    can do that, we might as well publish it. 

    Instead, we will generate multivariate normal data, but lower the SD so that
    negative datapoints are less likely. Our goal is to ensure that we don't 
    select a response type mean that is < 3 SDs away from 0. If we do so, we 
    limit the expected number of negative measurements to 2 per 1000 random
    generations - across all of our hormone metrics, this is fewer than 15 
    "completely bogus" values per 1000 patients which will hopefully hide within
    the noise of masking/measurement error. 

    Finally, for the other metrics not at risk of generating zeros, we want to 
    sample the means from a range within the bulk of the distribution (e.g. 
    middle 40%), so there is still some noise in the model that prevents overfitting 
    due to massive separation of the data (e.g. wildly different mean hormone
    metrics by patient response type that could be guessed at by inspection). 
    """
    generation_sd = sd/RESAMPLING_CONSTANT
    sampling_range_min = max(3*generation_sd, stats.norm.ppf(QUANTILE_WIDTH, mean, sd))
    sampling_range_max = mean + (mean-sampling_range_min)
    range_boundaries = np.linspace(stats.norm.cdf(sampling_range_min, mean, sd),\
                                   stats.norm.cdf(sampling_range_max, mean, sd),\
                                   hormone_num_ranges+1)
    print(range_boundaries)

    #Sample the means from each hormone range
    sampled_means = []
    for i in range(hormone_num_ranges):
      sampled_means.append(stats.norm.ppf(np.random.uniform(range_boundaries[i], \
                                                    range_boundaries[i+1]), mean, sd))
    #If there are fewer than 10 ranges to be sampled, set the remainder to be 
    #overall mean 
    if hormone_num_ranges < 10: 
      sampled_means = sampled_means + [mean for i in range(10-hormone_num_ranges)]
    
    #Set dictionary
    hormone_ranges[hormone] = sampled_means

Number of ranges for lh_fsh_ratio: 10
[0.3  0.34 0.38 0.42 0.46 0.5  0.54 0.58 0.62 0.66 0.7 ]
Number of ranges for estradiol: 7
[0.3        0.35714286 0.41428571 0.47142857 0.52857143 0.58571429
 0.64285714 0.7       ]
Number of ranges for testosterone: 9
[0.3        0.34444444 0.38888889 0.43333333 0.47777778 0.52222222
 0.56666667 0.61111111 0.65555556 0.7       ]
Number of ranges for progesterone_17oh: 6
[0.3        0.36666667 0.43333333 0.5        0.56666667 0.63333333
 0.7       ]
Number of ranges for dheas: 6
[0.46631374 0.47754249 0.48877125 0.5        0.51122875 0.52245751
 0.53368626]
Number of ranges for androstenedione: 8
[0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65 0.7 ]
Number of ranges for amh: 9
[0.3        0.34444444 0.38888889 0.43333333 0.47777778 0.52222222
 0.56666667 0.61111111 0.65555556 0.7       ]


In [6]:
#Examine in a dataframe format
response_type_means = pd.DataFrame(hormone_ranges)
response_type_means

,lh_fsh_ratio,estradiol,testosterone,progesterone_17oh,dheas,androstenedione,amh
0,1.404920,78.381501,59.774086,1.148508,692.231438,3.153781,58.182675
1,1.427874,81.527586,63.457611,1.325195,732.586451,3.978012,62.942082
2,1.516077,85.380144,63.606314,1.539597,755.636403,4.527994,69.142698
3,1.604467,88.912061,67.272326,1.626208,788.569311,4.834881,73.196544
4,1.679392,89.384372,70.851530,1.935247,817.303654,5.247471,75.973963
5,1.742142,92.776554,74.606542,2.119638,863.832703,6.023041,81.461009
6,1.836128,98.644721,79.051968,1.600000,777.500000,6.641497,83.597995
7,1.891633,87.600000,82.443333,1.600000,777.500000,7.050324,86.445893
8,1.915573,87.600000,86.009237,1.600000,777.500000,5.200000,91.698191
9,2.022716,87.600000,71.400000,1.600000,777.500000,5.200000,76.000000


In [7]:
#Looks good, but - we need to shuffle the values in each column independently 
#to obtain the final patient response type hormone means
for column in response_type_means.columns:
  means = response_type_means[column].values
  np.random.shuffle(means)
  response_type_means[column] = means

In [8]:
response_type_means

,lh_fsh_ratio,estradiol,testosterone,progesterone_17oh,dheas,androstenedione,amh
0,1.836128,78.381501,63.457611,2.119638,777.500000,4.834881,86.445893
1,1.516077,81.527586,71.400000,1.600000,777.500000,5.200000,83.597995
2,1.427874,92.776554,63.606314,1.935247,755.636403,6.641497,76.000000
3,1.404920,87.600000,59.774086,1.600000,788.569311,6.023041,91.698191
4,2.022716,88.912061,74.606542,1.325195,692.231438,5.247471,62.942082
5,1.742142,89.384372,70.851530,1.600000,817.303654,3.978012,75.973963
6,1.915573,87.600000,86.009237,1.600000,777.500000,4.527994,81.461009
7,1.679392,85.380144,79.051968,1.148508,732.586451,7.050324,58.182675
8,1.604467,87.600000,67.272326,1.626208,863.832703,5.200000,69.142698
9,1.891633,98.644721,82.443333,1.539597,777.500000,3.153781,73.196544


In [9]:
#Load in covariance matrix and prepare: 
cov = np.load("cov_matrix.npy")
cov = cov * (COVARIANCE_STRENGTH*(np.ones((7,7)) - np.identity(7)) + np.identity(7))
w, v = np.linalg.eig(cov)
sigmas = np.sqrt(w) * v #get covariance-corrected SDs
print(f"Number of negative covariances: {np.sum(np.where(cov < 0, 1, 0))} (of 49 total covariances)")

Number of negative covariances: 22 (of 49 total covariances)


In [10]:
#Prepare our calculations for determining symptom prevalence 
pcos_hormone_means = []
pcos_hormone_sds = []
for hormone, dist in pcos_hormones.items():
    mean, sd = dist
    pcos_hormone_means.append(mean)
    pcos_hormone_sds.append(sd)
pcos_hormone_means = np.array(pcos_hormone_means)
pcos_hormone_sds = np.array(pcos_hormone_sds)

In [11]:
#Generate lots of patients, one response-type at a time 
all_patients = {}
for i in range(10): 
  #Indexing column - for response type 
  response_type = np.repeat(i+1, 10000)[:, np.newaxis]

  means = response_type_means.loc[0].values
  seeds = np.random.normal(loc=0, scale=(1/RESAMPLING_CONSTANT), size=(means.shape[0], 10000))
  #Hormone values 
  values = (sigmas @ seeds + np.repeat(means[:, np.newaxis], 10000, axis=1)).T

  avg_zscores = (values - np.repeat(pcos_hormone_means[np.newaxis, :], 10000, axis=0))/\
  np.repeat(pcos_hormone_sds[np.newaxis, :], 10000, axis=0)
  #Avg hormone z-scores (based on overall PCOS means) for calculating symptom freq. 
  avg_zscores = np.average(avg_zscores, axis=1)

  """
  Symptom Bernoulli probabilities - base distributions as follows (mean +/- SD):
  - Irregular menstruation 0.8 +/- 0.5 
  - Ovarian cysts 0.75 +/- 0.5
  - Hirsutism 0.75 +/- 0.5
  - Acne 0.25 +/- 0.1
  - Anxiety 0.42 +/- 0.1
  - Depression 0.37 +/- 0.1

  """
  symptom_probs = np.concatenate((0.8+0.05*avg_zscores[:,np.newaxis], 
                                  0.75+0.05*avg_zscores[:,np.newaxis],
                                  0.75+0.05*avg_zscores[:,np.newaxis], 
                                  0.25+0.1*avg_zscores[:,np.newaxis],
                                  0.42+0.1*avg_zscores[:,np.newaxis],
                                  0.37+0.1*avg_zscores[:,np.newaxis]),
                                 axis=1)
  symptom_assignments = np.random.uniform(size=symptom_probs.shape)
  #Symptom binary variables 
  symptom_assignments = np.where(symptom_assignments < symptom_probs, 1, 0)

  """
  Successful treatment - treatments will be numbered 1-10 for now 
  The most frequent treatment for response type 1 will be OCP1; response type 2 -> OCP2, etc.
  """
  treatments = np.random.uniform(size=(10000,1))
  treatments = np.where(treatments < PROPORTION_TRUE_TREATMENT, i+1, np.random.randint(1, 11))


  columns = ['response_type'] + list(response_type_means.columns) + \
            ['irreg_menstr', 'cysts', 'hirsutism', 'acne', 'anxiety', 'depression', 'treatment']
  
  #Create data array, remove negative values 
  data_array = np.concatenate((response_type, values, symptom_assignments, treatments), axis=1)
  data_array = np.where(data_array < 0, 0, data_array)

  #Create dataframe for all patients of one response type 
  response_type_temp = pd.DataFrame(data_array)
  response_type_temp.columns = columns
  all_patients[i+1] = response_type_temp


In [12]:
###Generating 15 clinics - start by getting proportions of patient response types
#Use gamma distribution instead of random uniform to generate highly unequal proportions 
#of patient response types in different clinics
clinic_response_type_proportions = np.random.gamma(shape=1.5, scale=1.5, size=(15,10))
clinic_response_type_proportions /= np.repeat(np.sum(clinic_response_type_proportions, axis=1)[:, np.newaxis], 10, axis=1)

#Now, generate the number of patients in each of 15 clinics using random normal distribution
#Assume 1500 +/- 300 patients per practice
clinic_sizes = np.repeat(np.round(np.random.normal(loc=1500, scale=300, size=(15,1)), 0), 10, axis=1)

#Finally, do element-wise multiplication to find the number of patients per response type to sample per clinic
num_responsetypes_per_clinic = np.round(clinic_response_type_proportions * clinic_sizes, 0).astype('int')

In [13]:
num_responsetypes_per_clinic

array([[100, 479, 349, 173, 154,  97,  73,  28, 235, 154],
       [ 80, 195,  29, 152, 128, 203,  88,  39,  29, 303],
       [  4,  65,  49, 200,   9,  31,  62, 235,  23, 123],
       [ 33, 356, 232,  98,  84,  88,  69,  99, 151,  87],
       [ 13, 234, 191,  71,  37,  61,  18,  35,   3, 147],
       [128, 313, 230,  73,  82,  64, 122, 352, 338, 221],
       [ 73, 160, 157,  83, 248, 397, 212, 171,  68, 380],
       [210, 116, 159,  69, 166,  33, 407, 110, 112, 287],
       [ 62,   5, 270,  41, 435, 296,  84, 203, 679,  91],
       [  1, 165, 226, 202,  78,  52, 139, 183, 289,  41],
       [ 67, 590, 142, 132,  98, 143,  48,  45, 172,  26],
       [711, 100, 398,  75,  74, 101, 275, 163, 141, 318],
       [192,  76, 294, 268,  61, 120,  59, 187,  84, 432],
       [124,  51, 175, 304,  49, 166, 174,  47,  19,  60],
       [ 38, 297, 119,  37,  37,  71,  20, 193, 120,  41]])

In [14]:
#Because the response types are randomly generated, no need to reshuffle the data
clinics = {}
prev_idxs = np.zeros((10,)).astype('int')
for clinic in range(15):
  #Slice a row from the array specifying number of each patient response type for the clinic
  #This will specify how many more rows of each patient response type dataframe to sample 
  #and concatenate to form one clinic's worth of data
  next_idxs = prev_idxs + num_responsetypes_per_clinic[clinic]

  #Fetch rows from the appropriate response type-specific dataframes to make clinic df
  clinic_patients = None
  for i, response_type in enumerate(next_idxs):
    if clinic_patients is None:
      clinic_patients = all_patients[i+1].iloc[prev_idxs[i]:next_idxs[i]]
    else:
      clinic_patients = pd.concat([clinic_patients, all_patients[i].iloc[prev_idxs[i]:next_idxs[i]]], ignore_index=True)
  clinics[clinic] = clinic_patients
  #Set new index positions for the next round of row slicing
  prev_idxs = next_idxs

In [15]:
#Add noise per clinic
#Use gamma distribution to select "average percentage hormone noise" and 
#"average symptom masking probability"

clinic_hormone_percent_noise = PER_CLINIC_NOISE_SPREAD*np.random.gamma(shape=1, scale=2, size=(15))
clinic_symptom_mask_prob = SYMPTOM_MASK_CONST*np.random.gamma(shape=9, scale=0.5, size=(15))

for i in range(15):
  print(f"Clinic {i+1} will have mean 0, SD ({clinic_hormone_percent_noise[i]}% hormone mean) noise added to hormone metrics, and {clinic_symptom_mask_prob[i]}% of symptoms randomly masked")

Clinic 1 will have mean 0, SD (5.031680942936835% hormone mean) noise added to hormone metrics, and 25.184844067666177% of symptoms randomly masked
Clinic 2 will have mean 0, SD (0.8544373604643642% hormone mean) noise added to hormone metrics, and 15.997538036177303% of symptoms randomly masked
Clinic 3 will have mean 0, SD (2.2216478098203205% hormone mean) noise added to hormone metrics, and 18.7907921243227% of symptoms randomly masked
Clinic 4 will have mean 0, SD (0.18181991598514338% hormone mean) noise added to hormone metrics, and 16.85928281745749% of symptoms randomly masked
Clinic 5 will have mean 0, SD (0.5624629619128422% hormone mean) noise added to hormone metrics, and 21.688130995143947% of symptoms randomly masked
Clinic 6 will have mean 0, SD (1.5241589793469354% hormone mean) noise added to hormone metrics, and 22.944781943001086% of symptoms randomly masked
Clinic 7 will have mean 0, SD (0.36726567451239356% hormone mean) noise added to hormone metrics, and 19.6593

In [16]:
#Add noise per clinic
for clinic, patients in clinics.items():
  #For hormone measurements
  for hormone in ['lh_fsh_ratio', 'estradiol', 'testosterone', 'progesterone_17oh', 'dheas', 'androstenedione', 'amh']:
    measurements = patients[hormone].values 
    #Create noise vector for each hormone independently, add to hormone measurements 
    noise = np.random.normal(loc=0, scale=HORMONE_MEASUREMENT_NOISE*0.01*clinic_hormone_percent_noise[clinic]*np.mean(measurements), size=measurements.shape)
    patients[hormone] = np.where(measurements + noise < 0, 0, measurements + noise)
  #For symptoms
  for symptom in ['irreg_menstr', 'cysts', 'hirsutism', 'acne', 'anxiety', 'depression']:
    measurements = patients[symptom].values
    #Mask out symptoms at the clinic-specified rate 
    mask_prob = np.random.uniform(size=measurements.shape)
    patients[symptom] = np.where(mask_prob < 0.01*clinic_symptom_mask_prob[clinic], 0, measurements).astype('int')
  #Set patient response type label (DO NOT USE AS A FEATURE), treatment label as ints 
  patients['treatment'] = patients.treatment.values.astype('int')
  patients['response_type'] = patients.response_type.values.astype('int')
  patients.to_csv(f'clinic_datasets/clinic_{clinic}.csv')